# Scratch Pad

In [1]:
%matplotlib qt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import numpy as np

## 3D Model of annotations on human face

In [2]:
pts_3d = np.float32([[-7.308957,0.913869,0.000000],
[-6.775290,-0.730814,-0.012799],
[-5.665918,-3.286078,1.022951],
[-5.011779,-4.876396,1.047961],
[-4.056931,-5.947019,1.636229],
[-1.833492,-7.056977,4.061275],
[0.000000,-7.415691,4.070434],
[1.833492,-7.056977,4.061275],
[4.056931,-5.947019,1.636229],
[5.011779,-4.876396,1.047961],
[5.665918,-3.286078,1.022951],
[6.775290,-0.730814,-0.012799],
[7.308957,0.913869,0.000000],
[5.311432,5.485328,3.987654],
[4.461908,6.189018,5.594410],
[3.550622,6.185143,5.712299],
[2.542231,5.862829,4.687939],
[1.789930,5.393625,4.413414],
[2.693583,5.018237,5.072837],
[3.530191,4.981603,4.937805],
[4.490323,5.186498,4.694397],
[-5.311432,5.485328,3.987654],
[-4.461908,6.189018,5.594410],
[-3.550622,6.185143,5.712299],
[-2.542231,5.862829,4.687939],
[-1.789930,5.393625,4.413414],
[-2.693583,5.018237,5.072837],
[-3.530191,4.981603,4.937805],
[-4.490323,5.186498,4.694397],
[1.330353,7.122144,6.903745],
[2.533424,7.878085,7.451034],
[4.861131,7.878672,6.601275],
[6.137002,7.271266,5.200823],
[6.825897,6.760612,4.402142],
[-1.330353,7.122144,6.903745],
[-2.533424,7.878085,7.451034],
[-4.861131,7.878672,6.601275],
[-6.137002,7.271266,5.200823],
[-6.825897,6.760612,4.402142],
[-2.774015,-2.080775,5.048531],
[-0.509714,-1.571179,6.566167],
[0.000000,-1.646444,6.704956],
[0.509714,-1.571179,6.566167],
[2.774015,-2.080775,5.048531],
[0.589441,-2.958597,6.109526],
[0.000000,-3.116408,6.097667],
[-0.589441,-2.958597,6.109526],
[-0.981972,4.554081,6.301271],
[-0.973987,1.916389,7.654050],
[-2.005628,1.409845,6.165652],
[-1.930245,0.424351,5.914376],
[-0.746313,0.348381,6.263227],
[0.000000,0.000000,6.763430],
[0.746313,0.348381,6.263227],
[1.930245,0.424351,5.914376],
[2.005628,1.409845,6.165652],
[0.973987,1.916389,7.654050],
[0.981972,4.554081,6.301271]])

fig = plt.figure()
#ax = fig.add_subplot(111, projection='3d') #, aspect='equal'
ax = fig.gca(projection='3d')
ax.set_xlim3d(-10,10)
ax.set_ylim3d(-10,10)
ax.set_zlim3d(-10,10)

c = 0
for x, y, z in pts_3d:
    #print("%.2f, %.2f, %.2f" % (x, y, z))
    if c in [33,29,34,38,13,17,25,21,55,49,43,39,45,6]:
        ax.scatter(x, y, z, c='r', marker='o')
    else:
        ax.scatter(x, y, z, c='tab:orange', marker='o')
    c = c + 1
    
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
#ax.view_init(15, 15)

for angle in range(0, 360):
    ax.view_init(30, 40)

plt.show()

## Yawn Estimation Calibration

In [3]:
#%matplotlib qt
#import matplotlib.pyplot as plt
#import numpy as np
from scipy.optimize import curve_fit
from lmfit import Model
from lmfit.models import GaussianModel
import math
import re
from math import *
#from random import randint

### Yawn Datasets

In [4]:
big_yawn = np.array([[1129,0.113],[1260,0.125],[1415,0.125],[1563,0.139],[1716,0.127],[1857,0.134],[2024,0.129],[2164,0.185],[2323,0.406],[2520,0.445],[2729,0.531],[2963,0.631],[3126,0.639],[3307,0.667],[3480,0.667],[3672,0.704],[3826,0.763],[4018,0.801],[4202,0.824],[4353,0.811],[4519,0.832],[4721,0.871],[4875,0.848],[5067,0.935],[5226,0.949],[5403,0.912],[5560,0.893],[5795,0.808],[6031,0.691],[6218,0.493],[6407,0.319],[6561,0.215],[6726,0.123],[6900,0.109],[7074,0.113],[7275,0.130],[7450,0.113]])
med_yawn = np.array([[908,0.136],[1014,0.136],[1125,0.106],[1263,0.136],[1432,0.121],[1563,0.141],[1696,0.321],[1816,0.428],[1958,0.484],[2085,0.518],[2223,0.554],[2362,0.536],[2510,0.501],[2685,0.538],[2830,0.555],[2967,0.581],[3101,0.546],[3292,0.943],[3496,1.035],[3627,1.016],[3767,1.053],[3950,0.829],[4091,0.537],[4228,0.276],[4448,0.164],[4625,0.145],[4833,0.121],[5053,0.125],[5235,0.126],[5445,0.109],[5699,0.141],[5936,0.125],[6101,0.145],[6508,0.111]])
small_yawn = np.array([[840,0.206],[988,0.188],[1125,0.172],[1221,0.169],[1332,0.167],[1475,0.251],[1631,0.367],[1805,0.561],[1938,0.649],[2070,0.660],[2227,0.643],[2373,0.569],[2525,0.572],[2677,0.500],[2846,0.483],[2999,0.493],[3176,0.356],[3330,0.191],[3496,0.135],[3688,0.101],[3846,0.153],[4003,0.119],[4184,0.130],[4343,0.109],[4482,0.133],[4851,0.132]])
violent_yawn = np.array([[929,0.203],[1063,0.191],[1243,0.172],[1380,0.172],[1559,0.196],[1685,0.333],[1881,0.728],[2077,0.807],[2288,0.691],[2481,0.580],[2673,0.367],[2826,0.617],[3008,0.777],[3226,0.632],[3405,0.957],[3596,0.702],[3798,0.849],[3930,0.960],[4079,0.994],[4303,1.027],[4685,0.888],[4809,0.756]])
long_yawn = np.array([[859,0.139],[1020,0.132],[1170,0.116],[1294,0.103],[1484,0.127],[1634,0.167],[1774,0.291],[1908,0.350],[2047,0.367],[2184,0.368],[2314,0.400],[2462,0.406],[2639,0.408],[2789,0.412],[2935,0.500],[3105,0.619],[3259,0.675],[3489,0.760],[3650,0.819],[3846,0.827],[3994,0.793],[4171,0.848],[4323,0.798],[4515,0.954],[4646,0.974],[4841,1.003],[4998,0.957],[5181,0.892],[5395,0.970],[5577,0.970],[5786,0.934],[5989,0.947],[6192,0.521],[6378,0.353],[6578,0.271],[6714,0.284],[6867,0.261],[7044,0.282],[7181,0.260],[7385,0.221],[7572,0.191],[7796,0.122],[7978,0.131],[8172,0.134],[8323,0.145],[8535,0.141],[8716,0.136],[8902,0.123],[9056,0.110],[9241,0.135],[9453,0.128]])
short_yawn = np.array([[854,0.150],[990,0.108],[1117,0.109],[1249,0.126],[1365,0.112],[1499,0.153],[1626,0.164],[1755,0.404],[1913,0.436],[2092,0.421],[2224,0.501],[2356,0.501],[2493,0.447],[2620,0.380],[2769,0.128],[3003,0.121],[3174,0.106],[3351,0.119],[3494,0.108],[3641,0.132],[3834,0.109],[4018,0.107],[4149,0.122],[4306,0.119]])
jibberish = np.array([[924,0.101],[1085,0.122],[1209,0.119],[1365,0.118],[1525,0.121],[1673,0.152],[1828,0.284],[1987,0.450],[2189,0.329],[2349,0.176],[2501,0.344],[2724,0.269],[2886,0.392],[3110,0.319],[3342,0.132],[3559,0.167],[3698,0.236],[3854,0.180],[4002,0.188],[4224,0.517],[4450,0.124],[4665,0.413],[4822,0.355],[4980,0.155],[5187,0.409],[5352,0.173],[5586,0.198],[5806,0.169],[5964,0.325],[6182,0.230],[6341,0.291],[6549,0.185],[6770,0.179],[6921,0.307],[7143,0.410],[7520,0.114],[7675,0.116]])
still_mouth = np.array([[904,0.123],[1065,0.137],[1207,0.123],[1361,0.122],[1544,0.122],[1685,0.107],[1823,0.139],[2006,0.105],[2154,0.105],[2348,0.118],[2536,0.122],[2742,0.135],[2959,0.135],[3163,0.120],[3351,0.132],[3489,0.118],[3699,0.130],[3866,0.122],[4038,0.119],[4201,0.118],[4350,0.122],[4587,0.108],[4745,0.132],[4956,0.121],[5144,0.130],[5335,0.145],[5481,0.125],[5874,0.121],[5990,0.121]])
talking = np.array([[864,0.112],[993,0.123],[1111,0.114],[1252,0.112],[1391,0.098],[1551,0.129],[1725,0.395],[1910,0.302],[2092,0.180],[2254,0.370],[2448,0.172],[2641,0.165],[2849,0.264],[3042,0.208],[3256,0.137],[3440,0.219],[3585,0.130],[3771,0.113],[3967,0.250],[4110,0.287],[4291,0.344],[4476,0.263],[4653,0.229],[4836,0.133],[5042,0.308],[5245,0.533],[5445,0.319],[5654,0.268],[5830,0.250],[5989,0.153],[6164,0.146],[6359,0.221],[6529,0.223],[6698,0.129],[6902,0.117],[7095,0.173],[7322,0.104],[7498,0.112],[7657,0.102],[7851,0.129],[8034,0.097]])
singing = np.array([[919,0.224],[1100,0.192],[1239,0.167],[1437,0.228],[1608,0.273],[1775,0.207],[1972,0.674],[2123,0.635],[2310,0.437],[2513,0.163],[2720,0.298],[2909,0.336],[3111,0.214],[3308,0.228],[3504,0.553],[3701,0.532],[3898,0.291],[4034,0.227],[4236,0.230],[4438,0.182],[4637,0.724],[4857,0.735],[5042,0.688],[5240,0.654],[5441,0.257],[5649,0.136],[5830,0.205],[6019,0.188],[6162,0.266],[6366,0.315],[6569,0.423],[6751,0.355],[6956,0.307],[7176,0.132],[7366,0.127],[7559,0.116],[7698,0.116],[7894,0.118]])

### Gaussian Model Fitting Functions

In [5]:
def xy_to_x_and_y(xy):
    x = xy[:, 0]
    y = xy[:, 1]
    return x, y

def gaussian(x, amp, cen, wid):
    return (amp / (np.sqrt(2*pi) * wid)) * np.exp(-(x-cen)**2 / (2*wid**2))

def fit_to_gaussian(x, y, amp=2200, cen=4500, wid=1100):
    gmodel = GaussianModel() #= Model(gaussian)
    params = gmodel.guess(y, x=x)
    c = params['center'].value
    n = len(y)
    q3 = ((np.max(x) - c)/2) + c
    min_x = np.min(x)
    q1 = ((params['center'].value - min_x)/2) + min_x
    s = params['sigma'].value
    h = params['height'].value
    max_y = np.max(y)
    if np.max([h, max_y]) < 0.5:
        amp = 1/n
        diff_h = 0.6 - h
        gmodel.set_param_hint('amplitude', value=amp)
        gmodel.set_param_hint('amplitude', max=amp*(1+diff_h))
        gmodel.set_param_hint('amplitude', min=amp*diff_h)
    gmodel.set_param_hint('center', value=c)
    gmodel.set_param_hint('center', max=q3)
    gmodel.set_param_hint('center', min=q1)
    gmodel.set_param_hint('sigma', value=s)
    gmodel.set_param_hint('sigma', min=s/2)
    gmodel.set_param_hint('sigma', max=s*1.5)
    gmodel.set_param_hint('height', min=0.6) 
    result = gmodel.fit(y, x=x)
    #gmodel.print_param_hints()
    report = result.fit_report()
    chi_re = re.compile(r'chi-square\s+=\s+([0-9.]+)')
    cor_re = re.compile(r'C\(sigma, amplitude\)\s+=\s+([0-9.-]+)')
    chis = np.float32(chi_re.findall(report))
    cors = np.float32(cor_re.findall(report))
    coeffs = np.concatenate((chis,cors))
    mse_model = assess_fit(y, result.init_fit - result.best_fit)
    mse_yhat = assess_fit(y, result.residual)
    return [mse_model, mse_yhat, result, report, coeffs]

def plot_fit(x, y, result, title):
    plt.title(title)
    plt.plot(x, y, 'bo')
    plt.plot(x, result.init_fit, 'k--')
    plt.plot(x, result.best_fit, 'r-')
    plt.xlabel('Milliseconds')
    plt.ylabel('Mouth Aspect Ratio (MAR)')
    plt.show()
    
def assess_fit(y, residuals):
    n = len(y)
    RSS = 0
    for i in range(0, n):
        RSS += residuals[i]**2
    MSE = RSS/n
    return MSE

### Iterate Datasets to fit to Model

In [6]:
np.set_printoptions(formatter={'float': "{0:.3f}".format})
datasets = [big_yawn,med_yawn,small_yawn,violent_yawn,long_yawn,short_yawn,jibberish,still_mouth,talking,singing]
names=["Big Yawn","Med Yawn","Small Yawn","Violent Yawn","Long Yawn","Short Yawn","Jibberish","Still Mouth","Talking","Singing"]
for d in range(0,len(datasets)):
    x, y = xy_to_x_and_y(datasets[d])
    results = fit_to_gaussian(x, y) #, coeffs, report
    print(names[d])
    print("\tGoF:%.5f\tMSE:%.5f" % (results[0],results[1]))
    if results[0] > 0.2 and results[1] < 0.1:
        print("\tYAWN MODEL FITS")

Big Yawn
	GoF:0.33478	MSE:0.00855
	YAWN MODEL FITS
Med Yawn
	GoF:0.21420	MSE:0.02948
	YAWN MODEL FITS
Small Yawn
	GoF:0.13359	MSE:0.00527
Violent Yawn
	GoF:0.44466	MSE:0.02115
	YAWN MODEL FITS
Long Yawn
	GoF:0.29536	MSE:0.00836
	YAWN MODEL FITS
Short Yawn
	GoF:0.06189	MSE:0.00621
Jibberish
	GoF:0.05899	MSE:0.01108
Still Mouth
	GoF:0.00000	MSE:0.01531
Talking
	GoF:0.04273	MSE:0.00705
Singing
	GoF:0.10964	MSE:0.02818


### Fit one dataset at a time

In [7]:
x, y = xy_to_x_and_y(singing)
results = fit_to_gaussian(x, y) #, coeffs, report
result = results[2]
report = results[3]

#### Plot it

In [9]:
plot_fit(x, y, result, 'Singing')

#### Eval uncertainty

In [8]:
result.eval_uncertainty(sigma=3)

array([0.176, 0.179, 0.180, 0.181, 0.180, 0.178, 0.174, 0.170, 0.165,
       0.158, 0.151, 0.144, 0.138, 0.132, 0.128, 0.124, 0.122, 0.121,
       0.121, 0.122, 0.124, 0.128, 0.131, 0.135, 0.140, 0.144, 0.148,
       0.152, 0.154, 0.156, 0.158, 0.158, 0.157, 0.154, 0.150, 0.146,
       0.142, 0.135])

#### View report

In [11]:
print(report)

[[Model]]
    Model(gaussian)
[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 62
    # data points      = 38
    # variables        = 3
    chi-square         = 1.07082291
    reduced chi-square = 0.03059494
    Akaike info crit   = -129.628032
    Bayesian info crit = -124.715273
[[Variables]]
    sigma:      2450.99994 +/- 480.698813 (19.61%) (init = 1634)
    center:     4066.82513 +/- 333.017330 (8.19%) (init = 3709.556)
    amplitude:  2688.52517 +/- 368.895879 (13.72%) (init = 1)
    fwhm:       5771.66368 +/- 1131.95918 (19.61%) == '2.3548200*sigma'
    height:     0.60000000 +/- 0.05405447 (9.01%) == '0.3989423*amplitude/max(1.e-15, sigma)'
[[Correlations]] (unreported correlations are < 0.100)
    C(sigma, amplitude)  =  0.781
    C(sigma, center)     = -0.140
    C(center, amplitude) = -0.128



#### Assess fit

In [12]:
assess_fit(y,result.residual)

0.02817955031218037

#### Plot residuals

In [13]:
plt.plot(x, result.residual, 'bo')
plt.show

<function matplotlib.pyplot.show(*args, **kw)>

#### Plot Uncertainty

In [14]:
dely = result.eval_uncertainty(sigma=3)
plt.fill_between(x, result.best_fit-dely, result.best_fit+dely, color="#ABABAB")